In [2]:
import os
import sys
import utils
import random
import dataset
import binvox_rw
import numpy as np
import pandas as pd
from PIL import Image
from datetime import datetime
import matplotlib.pyplot as plt
import tensorflow as tf

#import tensorflow.contrib.eager as tfe
#tfe.enable_eager_execution()
sess=tf.InteractiveSession()
# load data
shapenet=dataset.ShapeNet()
#shapenet.batch_size=3

/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
with tf.name_scope('input'):
    x=tf.placeholder(tf.float32,[shapenet.batch_size,137,137,3])
    y=tf.placeholder(tf.float32,[shapenet.batch_size,32,32,32])

In [4]:
# encoder network
cur_tensor=x
print(cur_tensor.shape)
k_s = [3,3]
conv_filter_count = [96, 128, 256, 256, 256, 256]
for i in range(6): 
    ks=[7,7]if i is 0 else k_s  
    with tf.name_scope("encoding_block"):
        cur_tensor=tf.layers.conv2d(cur_tensor,filters=conv_filter_count[i],padding='SAME',kernel_size= k_s,activation=None)
        cur_tensor=tf.layers.max_pooling2d(cur_tensor,2,2)
        cur_tensor=tf.nn.relu(cur_tensor)
        print(cur_tensor.shape)

# flatten tensor
cur_tensor=tf.contrib.layers.flatten(cur_tensor)
cur_tensor=tf.contrib.layers.fully_connected(cur_tensor,1024,activation_fn=None)
print(cur_tensor.shape)

(24, 137, 137, 3)
(24, 68, 68, 96)
(24, 34, 34, 128)
(24, 17, 17, 256)
(24, 8, 8, 256)
(24, 4, 4, 256)
(24, 2, 2, 256)
(24, 1024)


In [5]:
# 3D_Convolutional_LSTM 

with tf.name_scope("3D_LSTM"):
    #x=tf.placeholder(tf.float32,[None,1024])
    h_t=tf.Variable(tf.zeros([4,4,4,1,256]),name="hidden_state")
    s_t=tf.Variable(tf.zeros([4,4,4,1,256]),name="cell_state")

    # W,U,b for f,i,s,o
    W=[tf.Variable(tf.zeros([1024,256]),name="W%d"%(i)) for i in range(4)]
    U=[tf.Variable(tf.zeros([3,3,3,256,256]),name="U%d"%(i)) for i in range(4)]
    b=[tf.Variable(tf.zeros([1,256]),name="b%d"%(i)) for i in range(4)]

    Wx=[tf.matmul(cur_tensor, W[i]) for i in range(4)]
    Uh=[tf.nn.convolution(h_t,U[i],padding="SAME") for i in range(4)]


    f_t=tf.sigmoid(Wx[0] + Uh[0] + b[0])
    i_t=tf.sigmoid(Wx[1] + Uh[1] + b[1])
    o_t=tf.sigmoid(Wx[2]+ Uh[2] + b[2])
    s_t=tf.multiply(f_t,s_t) + tf.multiply(i_t,tf.tanh(Wx[3]+ Uh[3] + b[3]))
    h_t=tf.tanh(s_t)

    cur_tensor=tf.transpose(h_t,[3,0,1,2,4])
    print(cur_tensor.shape)

(24, 4, 4, 4, 256)


In [6]:
# decoding network
# batch_size=cur_tensor.shape.as_list()[0]
# cur_tensor=tf.reshape(cur_tensor,[batch_size,2,2,2,-1])
# print(cur_tensor.shape)
k_s = [3,3,3]
deconv_filter_count = [128, 128, 128, 64, 32, 2]
cur_tensor=utils.unpool(cur_tensor)
print(cur_tensor.shape)
for i in range(2,4): 
    with tf.name_scope("decoding_block"):
            cur_tensor=tf.layers.conv3d_transpose(cur_tensor,padding='SAME',filters=deconv_filter_count[i],kernel_size= k_s,activation=None)
            cur_tensor=utils.unpool(cur_tensor)
            cur_tensor=tf.nn.relu(cur_tensor)
            print(cur_tensor.shape)
            
for i in range(4,6): 
    with tf.name_scope("decoding_block_without_unpooling"):
            cur_tensor=tf.layers.conv3d_transpose(cur_tensor,padding='SAME',filters=deconv_filter_count[i],kernel_size= k_s,activation=None)
            cur_tensor=tf.nn.relu(cur_tensor)
            print(cur_tensor.shape)

y_hat=cur_tensor

(24, 8, 8, 8, 256)
(24, 16, 16, 16, 128)
(24, 32, 32, 32, 64)
(24, 32, 32, 32, 32)
(24, 32, 32, 32, 2)


In [11]:
# start new or continue exisiting training session
cur_time=str(datetime.now())
cur_time=cur_time.replace(" ","_")
cur_time=cur_time.replace(":","-")
    
root_dir="train_dir"
if not os.path.isdir(root_dir):
    os.makedirs(root_dir)

dir_list=os.listdir(root_dir)
if dir_list:
    max_dir=""
    for cur_dir in dir_list:
        if cur_dir>max_dir:    
            max_dir=cur_dir
    train_dir=max_dir
    print("continuing training session {}".format(train_dir))
else:
    print("new training session")
    train_dir=os.path.join(root_dir,cur_time)
    os.makedirs(train_dir)
    

saver = tf.train.Saver()

# train network
epoch=5
tf.global_variables_initializer().run()


#3d voxel-wise softmax
y_hat=tf.nn.softmax(y_hat)
p=y_hat[:,:,:,:,0]
q=y_hat[:,:,:,:,1]
loss_voxel=tf.reduce_mean(tf.reduce_sum(tf.multiply(tf.log(p),y)+tf.multiply(tf.log(q),1-y),[1,2,3]))
optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss_voxel)

loss=[]       
for e in range(1,epoch+1): 
    common_dir="./{}/epoch_{:03d}/batch_{:03d}".format(train_dir,e,0)
    if not os.path.isdir(common_dir):
        os.makedirs(common_dir)
        save_path = saver.save(sess,"{}/model.ckpt".format(common_dir))
        print("inital config for epoch saved at {}/epoch_{:03d}/batch_{:03d}".format(train_dir,e,0))
    else:
        print("possibliy corrupt intial param config manaully create inital condition")
    batch_number=1
    train=shapenet.next_train_batch()
    while(train is not None):
        common_dir="./{}/epoch_{:03d}/batch_{:03d}".format(train_dir,e,batch_number)
        if not os.path.isdir(common_dir):
            train_X=dataset.load_dataset(train[:,0])
            train_Y=dataset.load_labels(train[:,1])
            fd={x:train_X , y: train_Y};
            l=sess.run([loss_voxel],feed_dict=fd)
            loss.append(l) 
            os.makedirs(common_dir)
            # record parameters and generate plots
            fig = plt.figure()
            plt.plot(loss)
            plt.savefig("{}/loss.png".format(common_dir),bbox_inches='tight')
            save_path = saver.save(sess,"{}/model.ckpt".format(common_dir))
            plt.close()
            print("{}: epoch {:03d} batch {:03d} ".format(train_dir,e,batch_number))
        else:
            prev_dir="./{}/epoch_{:03d}/batch_{:03d}".format(train_dir,e,batch_number-1)
            saver.restore(sess,"{}/model.ckpt".format(prev_dir))
            continue
        
        # update
        train=shapenet.next_train_batch()
        batch_number+=1

new training session
inital config for epoch saved at train_dir/2017-12-29_16-55-48.403157/epoch_001/batch_000
train_dir/2017-12-29_16-55-48.403157: epoch 001 batch 001 
train_dir/2017-12-29_16-55-48.403157: epoch 001 batch 002 
train_dir/2017-12-29_16-55-48.403157: epoch 001 batch 003 
train_dir/2017-12-29_16-55-48.403157: epoch 001 batch 004 
train_dir/2017-12-29_16-55-48.403157: epoch 001 batch 005 
train_dir/2017-12-29_16-55-48.403157: epoch 001 batch 006 
train_dir/2017-12-29_16-55-48.403157: epoch 001 batch 007 
train_dir/2017-12-29_16-55-48.403157: epoch 001 batch 008 
train_dir/2017-12-29_16-55-48.403157: epoch 001 batch 009 


KeyboardInterrupt: 

In [ ]:
# tensorboard
writer = tf.summary.FileWriter("./logs/")
writer.add_graph(sess.graph)

In [ ]:
# model output as a voxelized image
out=tf.cast(tf.argmax(y_hat,axis=4),dtype=tf.float32)
out=sess.run(out,feed_dict=feed_dict)
outvoxel=binvox_rw.Voxels(out,out.shape,[0,0,0],1,'xzy')
with open("out/{i}.binvox",'w') as f:
    outvoxel.write(f)